In [557]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.quantum_info import Statevector, Operator, partial_trace
from qiskit.circuit.library.standard_gates import RYGate, RZGate
import matplotlib.pyplot as plt
import numpy as np
import pylatexenc

## Quantum State Preparation

Takes a $2^n$ dimensional vector $\ket{\psi}\in\mathbb{C}^{2^n}$ such that $||\psi||_2=1$, and outputs a circuit $U$ such that:

$$U\ket{0}_n = \sum_{x=0}^{2^n-1}\psi_x\ket{x}_n$$

It does this by iteratively building states:

$$\ket{\psi} = \sum_{x\in\mathbb{F_2^n}}\psi_{x0}\ket{x}_{n-1}\ket{0} + \psi_{x1}\ket{x}_{n-1}\ket{1}$$

So in order to prepare state $\ket{\psi}_n$, we must first prepare state $\ket{\psi}_{n-1}$, $\ket{\psi}_{n-2}$, $\dots$, $\ket{\psi}_{1}$. 

Here we demonstrate how the code works for the toy example $n=3$, with:
$$\ket{\psi}_3 = a\ket{000} + b\ket{100} + c\ket{010} + d\ket{110} + e\ket{001} + f\ket{101} + g\ket{011} + h\ket{111}$$

This state is entered into the main function *make_circuit* as a vector $[a, b, c, d, e, f, g, h]$.

- Get angles
    - Call subfunction *get_angles* on $\ket{\psi}_3$. This references every other entry of the vector (in this case, $a, c, e, g$) in order to build the angles of rotation:
        - $\theta_{200} = 2\arccos{\frac{a}{\sqrt{a^2+b^2}}}$
        - $\theta_{210} = 2\arccos{\frac{c}{\sqrt{c^2+d^2}}}$
        - $\theta_{201} = 2\arccos{\frac{e}{\sqrt{e^2+f^2}}}$
        - $\theta_{211} = 2\arccos{\frac{g}{\sqrt{g^2+h^2}}}$
    
        It also builds the state $\ket{\psi}_2 = \sqrt{a^2+b^2}\ket{00} + \sqrt{c^2+d^2}\ket{10} + \sqrt{e^2+f^2}\ket{01} + \sqrt{g^2+h^2}\ket{11}$, the previous step in the iterative process.

        It returns both the list of angles as well as this new state.

    - Call subfunction *get_angles* on $\ket{\psi}_2$ to get angles of rotation:
        - $\theta_{10} = 2\arccos{\frac{\sqrt{a^2+b^2}}{\sqrt{a^2+b^2+c^2+d^2}}}$
        - $\theta_{11} = 2\arccos{\frac{\sqrt{e^2+f^2}}{\sqrt{e^2+f^2+g^2+h^2}}}$

        And $\ket{\psi}_1 = \sqrt{a^2+b^2+c^2+d^2}\ket{0} + \sqrt{e^2+f^2+g^2+h^2}\ket{1}$

    - Call subfunction *get_angles* on $\ket{\psi}_1$ to get angle of rotation:
        - $\theta_0 = 2\arccos{\sqrt{a^2+b^2+c^2+d^2}}$

- Make list of basis states
    - Call the *make_list* subfunction to make a list of all the $1,...,n-2,n-1$ basis states. For this toy example, this gives: $[[0], [0, 1], [00, 01, 10, 11]]$.

    This list and its sublists correspond to the qubits associated with the $\theta$ angles of rotation. For instance, the rotation of the $\theta_{210}$ angle is associated with the $01$ value in the list. Since the first qubit of this is a zero, we apply an X gate to that qubit, apply a multi-controlled Y gate (controlled on both these qubits, and applied to the last qubit), and then uncompute the previously applied X gate.

- Make the circuit
    - Initialize the circuit $U$ on $n$ qubits.
    - Apply a Y rotation on the last qubit.

    - Run through the list of angles and corresponding basis states, apply an X gate if a qubit is zero, apply multi-controlled Y gates to the qubits, and then uncompute the X gate.

    
    



In [ ]:
def make_circuit(psi):
    """
    Takes a vector |ψ> and builds a circuit that will prepare that state.
    
    Args:
    - psi: A vector input of a desired state |ψ>
    
    Returns:
    - U_circ: The quantum circuit that prepares the state |ψ>
     """

    def make_list(L,n1):
        bitslist = []
        for j1 in range(L):
            bitslist.append(format(j1,'0'+str(n1)+'b'))
        return bitslist

    def get_angles(state_0): #Finds the rotation angles of a given state. Outputs a list of angles as well as a new state
        angles = []
        newstate = []
        for j1 in range(len(state_0)//2):
            angles.append(2*np.arccos(state_0[2*j1]/np.sqrt(state_0[2*j1]**2+state_0[2*j1+1]**2)))
            newstate.append(np.sqrt(state_0[2*j1]**2+state_0[2*j1+1]**2))
        return angles, newstate

    L = len(psi)
    n = int(np.log2(L))

    #Iteratively build a list of angles of rotation
    #Get the angles of rotation to be used lastly in the iterative building process, as well as the coefficients of the state |ψ>_{n-1} (the state we must prepare in order to iteratively build |ψ>_n)
    tot_angles=[] #Start with an empty list
    [angles,newstate] = get_angles(psi)
    tot_angles.append(angles) #Add these angles to the list
    for j1 in range(n-1):
        #Get the angles of rotation to be used on the new state, as well as the coefficients of the next state down in the iterative process
        [angles,newstate] = get_angles(newstate)
        tot_angles.append(angles) #Add these angles to the list
    
    #We will perform this iterative process starting from |ψ>_1 and going up to state |ψ>_n, so we must reverse the order of the angles in the list.
    tot_angles = list(reversed(tot_angles))


    #Make a list of the basis states for n-1,...,1 (example for n = 3: [[00, 01, 10, 11] , [0, 1], [0]]). This is how we will determine where to put X gates in the circuit
    basis_states = []
    for j1 in range(1,n+1):
        basis_states.append(make_list(2**(n-j1),n-j1))

    #Again, since we are performing this iterative process starting with |ψ>_1 and going up to state |ψ>_n, we reverse the order of this list.
    basis_states=list(reversed(basis_states))

    #Initialize the circuit.
    q_reg = QuantumRegister(n, name = "x") #We need as many bits as there are in |ψ>_n
    U_circ = QuantumCircuit(q_reg, name = "Quantum State Preparation")

    #Perform the first Y rotation on the first qubit
    U_circ.ry(tot_angles[0][0],q_reg[n-1])
    
    #For the remainder of the rotations, we will go through the list of angles and apply them to the circuit with a controlled Y rotation gate.
    for j1 in range(1,len(tot_angles)):
        curr_angles = tot_angles[j1]
        for j2 in range(len(curr_angles)):
            now_angle = curr_angles[j2]

            #If the state corresponding to this angle has a zero qubit in it, we apply an X gate to that qubit both before and after the rotation, to ensure that the Y rotation applies correctly.
            for j3 in range(len(basis_states[j1][j2])):
                if basis_states[j1][j2][j3] == '0':
                    U_circ.x(q_reg[n-1-j3])

            #Multi-controlled Y gate
            U_circ.append(RYGate(now_angle).control(j1),q_reg[::-1][0:j1+1])

            #Uncompute previously applied X gates
            for j3 in range(len(basis_states[j1][j2])):
                if basis_states[j1][j2][j3] == '0':
                    U_circ.x(q_reg[n-1-j3])

    return U_circ

In [573]:
#Demonstration of the code for the case where n = 3
n = 3
psi = (1 + np.arange(2**n)) / np.linalg.norm(1 + np.arange(2**n))
print('|ψ>:',psi,'\n||ψ||_2:', np.linalg.norm(psi))

qc=make_circuit(psi)
qc.draw()

|ψ>: [0.070014   0.14002801 0.21004201 0.28005602 0.35007002 0.42008403
 0.49009803 0.56011203] 
||ψ||_2: 0.9999999999999999


┌────────────┐»
x_0: ─────────────────────────────────────────────────────────┤ Ry(2.2143) ├»
                        ┌────────────┐     ┌────────────┐┌───┐└─────┬──────┘»
x_1: ───────────────────┤ Ry(2.3005) ├─────┤ Ry(1.8743) ├┤ X ├──────■───────»
     ┌────────────┐┌───┐└─────┬──────┘┌───┐└─────┬──────┘├───┤      │       »
x_2: ┤ Ry(2.3545) ├┤ X ├──────■───────┤ X ├──────■───────┤ X ├──────■───────»
     └────────────┘└───┘              └───┘              └───┘              »
«               ┌────────────┐     ┌────────────┐     ┌────────────┐
«x_0: ──────────┤ Ry(1.8546) ├─────┤ Ry(1.7521) ├─────┤ Ry(1.7039) ├
«     ┌───┐     └─────┬──────┘┌───┐└─────┬──────┘┌───┐└─────┬──────┘
«x_1: ┤ X ├───────────■───────┤ X ├──────■───────┤ X ├──────■───────
«     ├───┤┌───┐      │       ├───┤      │       └───┘      │       
«x_2: ┤ X ├┤ X ├──────■───────┤ X ├──────■──────────────────■───────
«     └───┘└───┘              └───┘

In [576]:
#Get the state vector of the circuit and compare it with the |ψ> we input
st=Statevector(qc)
print('Error between |ψ> and output of the circuit:',np.linalg.norm(st.data - psi))

Error between |ψ> and output of the circuit: 2.0955000055363631e-16
